In [32]:
# External libraries
import numpy as np
import time
import matplotlib.pyplot as plt
import matplotlib.style as style
import pandas as pd
import astropy
import scipy
from filterpy.kalman import KalmanFilter 
from filterpy.common import Q_discrete_white_noise
from scipy.linalg import block_diag
from astropy import units as u
from poliastro.bodies import Earth, Mars, Sun, Moon
from poliastro.twobody import Orbit
from poliastro.plotting import OrbitPlotter2D
from poliastro.plotting import OrbitPlotter3D
import glob
# Own Libraries
from utility.utils import *
from KalmanFilter.kf import *
from Detect.detector import *
from Match.pair import *
from Match.icp import *

%matplotlib tk
style.use('seaborn-paper')

In [4]:
# LOAD ALL IMAGES:
dt = 10
dict = {}
for img in glob.glob(f"DATA/ephemeris sat/inclination zero/{dt} step size/*"):
    txt = img             # stringa
    t = txt.split('_')[1] # numero
    dict[t] = txt

In [9]:
df = pd.read_csv('DATA/H_L_combined.csv')
craters_cat = CatalogSearch(H=df, lat_bounds=[-2,2], lon_bounds=[-150,-145], CAT_NAME='COMBINED')
deg2km = 2*np.pi*1737.4/360
km2deg = 1/deg2km
craters_cat['Diam']*=0.5*km2deg
craters_cat.rename( columns={'Diam':'r_deg'}, inplace=True)


craters_cat_m = np.array(craters_cat)
triplets_cat_m = find_all_triplets(craters_cat_m)
triplets_cat = pd.DataFrame(triplets_cat_m, columns=['Angle1','Angle2','Angle3','des1','des2','des3','lon1','lat1','r1','lon2','lat2','r2','lon3','lat3','r3'])
triplets_cat

 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 


,Angle1,Angle2,Angle3,des1,des2,des3,lon1,lat1,r1,lon2,lat2,r2,lon3,lat3,r3
0,131.702158,36.468967,11.828875,3.981607,4.584064,1.302557,-145.956117,-1.955073,0.383687,-149.519503,-0.883337,0.483438,-146.567992,-1.136915,0.615915
1,53.599676,32.771921,93.628403,6.950379,2.999944,25.004154,-145.956117,-1.955073,0.383687,-149.519503,-0.883337,0.483438,-148.951853,1.553883,0.085623
2,50.045955,33.354983,96.599062,7.115080,2.980638,10.383449,-145.956117,-1.955073,0.383687,-149.519503,-0.883337,0.483438,-149.049604,1.743955,0.219861
3,80.335167,21.402663,78.262170,6.331374,2.911405,6.722846,-145.956117,-1.955073,0.383687,-149.519503,-0.883337,0.483438,-148.862726,0.327445,0.206019
4,88.975541,43.732452,47.292007,5.214857,3.987939,8.781075,-145.956117,-1.955073,0.383687,-149.519503,-0.883337,0.483438,-147.303944,0.424471,0.143797
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4891,29.893350,139.896036,10.210615,6.460663,17.923202,15.148044,-147.209019,-1.791135,0.107286,-149.892463,-1.318375,0.115720,-146.370661,-1.304916,0.096573
4892,179.095926,0.400038,0.504036,13.185585,11.321707,0.780619,-147.209019,-1.791135,0.107286,-149.892463,-1.318375,0.115720,-148.703249,-1.517117,0.134153
4893,156.822185,10.817131,12.360684,13.017015,11.532394,1.152894,-147.209019,-1.791135,0.107286,-149.892463,-1.318375,0.115720,-148.690824,-1.812486,0.169625
4894,165.344427,4.040576,10.614996,14.620558,10.005324,3.230954,-147.209019,-1.791135,0.107286,-149.892463,-1.318375,0.115720,-149.182164,-1.585453,0.130440


In [6]:
idx = 0
# IMG1:
filename = dict[str(idx+1)]
img=cv2.imread(filename)
craters_det = detect(img)

# Pandas DataFrame:
df_craters_det = sort_mat(craters_det)
# Find all triplets:
triplets = find_all_triplets(craters_det)
triplets_det= pd.DataFrame(triplets, columns=['Angle1','Angle2','Angle3','des1','des2','des3','x1','y1','r1','x2','y2','r2','x3','y3','r3'])

 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 


In [122]:
def inner_join(QUERY1,QUERY2, tol1, tol2):
    DFs, items = [], []
    for row in QUERY1.itertuples():
            Angle1_Q1, Angle2_Q1, Angle3_Q1 = row[0],row[1],row[2] 
            d1_Q1, d2_Q1, d3_Q1 = row[3], row[4], row[5]


            SEARCH = QUERY2[((abs(QUERY2.Angle1 - Angle1_Q1) < tol1)&(abs(QUERY2.Angle2-Angle2_Q1)< tol1)&(abs(QUERY2.Angle3-Angle3_Q1) < tol1))   \
        |  ((abs(QUERY2.Angle1 - Angle2_Q1) < tol1) & (abs(QUERY2.Angle2 - Angle3_Q1) < tol1) & (abs(QUERY2.Angle3 - Angle1_Q1) < tol1)) \
        |  ((abs(QUERY2.Angle1 - Angle3_Q1) < tol1) & (abs(QUERY2.Angle2 - Angle1_Q1) < tol1) & (abs(QUERY2.Angle3 - Angle2_Q1) < tol1))]

            SEARCH =   SEARCH[ ((abs(QUERY2.des1- d1_Q1) < tol2) & (abs(QUERY2.des2- d2_Q1) < tol2) & (abs(QUERY2.des3- d3_Q1)< tol2)) \
                             | ((abs(QUERY2.des1- d2_Q1) < tol2) & (abs(QUERY2.des2- d3_Q1) < tol2) & (abs(QUERY2.des3- d1_Q1)< tol2)) \
                             | ((abs(QUERY2.des1- d3_Q1) < tol2) & (abs(QUERY2.des2- d1_Q1) < tol2) & (abs(QUERY2.des3- d2_Q1)< tol2)) ]

            if SEARCH.shape[0]!=0:
                items.append(row)
                DFs.append(SEARCH)
    return DFs, items

def drop_join(join):

    def cond(join_dropped, row, cond_param):
        return np.any( join_dropped.Angle1 == row[cond_param])


    first = True
    for ind, row in joins[0].iterrows():
        if first:
            join_dropped = row
            first = False
        elif cond(join_dropped,row, 'Angle1')| cond(join_dropped,row, 'Angle2') | cond(join_dropped,row, 'Angle3'):
            pass
        else:
            join_dropped = pd.concat([join_dropped, row])
    return join_dropped


In [98]:
t1 = time.time()
QUERY1 = triplets_cat
QUERY2 = triplets_det
joins, items = inner_join(QUERY1, QUERY2, 3,2)

t2 = time.time()
deltat = t2-t1

In [123]:
joins[0]
join_dropped = drop_join(joins[0])
join_dropped

Angle1      6.405361
Angle2    160.672387
Angle3     12.922252
des1        4.193688
des2        2.443138
des3        5.189089
x1        506.000000
y1        739.000000
r1         20.500000
x2        735.500000
y2        704.500000
r2        125.000000
x3         49.000000
y3        652.000000
r3         74.000000
Name: 2984, dtype: float64

In [119]:
arg = items[0][0]
QUERY1.iloc[arg:arg+1]

,Angle1,Angle2,Angle3,des1,des2,des3,lon1,lat1,r1,lon2,lat2,r2,lon3,lat3,r3
7,161.210649,15.147464,3.641887,3.851721,4.644868,4.920012,-145.956117,-1.955073,0.383687,-149.519503,-0.883337,0.483438,-146.689675,-1.934686,0.15827
